### 读取数据

In [1]:
import os
import re

data_dir=r'D:\项目资料\基因表达\通路分析'
file_gene_path=os.path.join(data_dir, 'gene_alias.txt')
file_wnt4_path=os.path.join(data_dir, '(cancer) AND (WNT4)文献.txt')
file_ca_path = os.path.join(data_dir, '(cancer) AND (Carbonic Anhydrase 1)文献.txt')

def read_gene_data_file(file_path):
    data = []
    whole_line = ''
    with open(file_path) as f:
        for line in f.readlines():
            line = line[:-1]
            whole_line = whole_line + line
            if not line.endswith(' '):
                data.append(whole_line)
                whole_line = ''
        data.append(whole_line)
    return data

def sel_item(gene_data):
    result, item = [], []
    start_nbr, i = 1, 0
    while i < len(gene_data):
        if gene_data[i].startswith('%s.' % start_nbr):
#             print('start nbr: %s match!' % start_nbr)
            if len(item) > 0:
                result.append(item)
                item = []
            
            start_nbr = start_nbr + 1
        item.append(gene_data[i])
        i = i + 1
        
    if len(item) > 0:
        result.append(item)
            
    print(len(result))
    return result

def proc_item(item_arr):
    results = []
    for item_idx, item in enumerate(item_arr):
        starts_search_flag = 'Author information:'
        ends_search_flag = '(^DOI:)|(^©)|(^PMID: )|(^Copyright)'
        # 标题
        result_item = [item[2]]
        for idx, line_str in enumerate(item):
            # 找Author Information
            if re.search(starts_search_flag, line_str):
                # 找空行
                no = idx + 1
                while not item[no] == '':
                    no = no + 1
                no = no + 1
                
                # 摘要前可能有Comment in
                if item[no].startswith('Comment in') or item[no].startswith('Erratum in'):
                    while not item[no] == '':
                        no = no + 1
                    no = no + 1
                
                # 找摘要
                while not item[no] == '':
                    result_item.append(item[no])
                    no = no + 1
                    
                results.append(result_item)
                break
                
            # 找到结束记号
            if re.search(ends_search_flag, line_str):
                # 找空行
                ends_no = idx - 1
                starts_no = idx - 2
                while not item[starts_no] == '':
                    starts_no = starts_no - 1
                    
                # 写内容
                result_item.extend(item[starts_no + 1:ends_no])
                
                results.append(result_item)
                break
                
            
    return results


In [2]:
import random
def write_split_lines(data, train_path, val_path):
    results = []
    for item in data:
        for line in item:
            if len(line.strip()) > 10:
                results.append(line)
    
    random.shuffle(results)
    train_len = int(len(results) * 0.8)
    train_data, val_data = results[:train_len], results[train_len:]
    with open(train_path, 'w') as f:
        for line in train_data:
            f.write('%s\n' % line)
            
    with open(val_path, 'w') as f:
        for line in val_data:
            f.write('%s\n' % line)


In [4]:
base_dir = r'D:\projects\NLP\Gene\Data'
files = os.listdir(base_dir)
files = [file for file in files if '文献' in file]
files.remove('(cancer) AND (WNT4)文献.txt')
files.remove('(cancer) AND (Carbonic Anhydrase 1)文献.txt')

data = []
for file in files:
    print(file)
    data.extend(sel_item(read_gene_data_file(os.path.join(base_dir, file))))
write_split_lines(data, os.path.join(base_dir, 'train.txt'), os.path.join(base_dir, 'val.txt'))

(cancer) AND (c-myc)文献19810409-20040827.txt
9052
(cancer) AND (c-myc)文献20040827-20160827.txt
9757
(cancer) AND (c-myc)文献20160827-20220827.txt
8089
(cancer) AND (P53)文献19790501-19970827.txt
8217
(cancer) AND (P53)文献19970827-20000827.txt
7429
(cancer) AND (P53)文献20000827-20030827.txt
7938
(cancer) AND (P53)文献20030827-20060827.txt
8142
(cancer) AND (P53)文献20060827-20090827.txt
8454
(cancer) AND (P53)文献20090827-20120827.txt
9746
(cancer) AND (P53)文献20120827-20140827.txt
7483
(cancer) AND (P53)文献20140827-20160827.txt
7515
(cancer) AND (P53)文献20160827-20180827.txt
7459
(cancer) AND (P53)文献20180827-20200827.txt
7667
(cancer) AND (P53)文献20200827-20220827.txt
7601
(cancer) AND (STAT)文献19470201-20150827.txt
7479
(cancer) AND (STAT)文献20150827-20220827.txt
5687
(cancer) AND PDL1)文献20001002-20191028.txt
6721
(cancer) AND PDL1)文献20191008-20220827.txt
9872


In [5]:
data[0]

['1. Adv Exp Med Biol. 2020;1248:33-59. doi: 10.1007/978-981-15-3266-5_3.',
 '',
 'Roles of PD-1/PD-L1 Pathway: Signaling, Cancer, and Beyond.',
 '',
 'Ai L(1), Xu A(2), Xu J(3).',
 '',
 'Author information:',
 '(1)Department of Medical Oncology, Zhongshan Hospital, Fudan University, Shanghai, 200032, China. storysparrow@126.com.',
 '(2)Department of Rheumatology, Renji Hospital, Shanghai Jiaotong University, Shanghai, 200001, China.',
 '(3)Institutes of Biomedical Sciences, Zhongshan-Xuhui Hospital, Fudan University, Shanghai, 200032, China. jie_xu@fudan.edu.cn.',
 '',
 'Immunotherapies that target PD-1/PD-L1 axis have shown unprecedented success in a wide variety of human cancers. PD-1 is one of the key coinhibitory receptors expressed on T cells upon T cell activation. After engagement with its ligands, mainly PD-L1, PD-1 is activated and recruits the phosphatase SHP-2 in proximity to T cell receptor (TCR) and CD28 signaling. This event results in dephosphorylation and attenuation o

In [2]:
# data_wnt4 = proc_item(sel_item(read_gene_data_file(file_wnt4_path)))
# data_ca = proc_item(sel_item(read_gene_data_file(file_ca_path)))
data_wnt4 = sel_item(read_gene_data_file(file_wnt4_path))
data_ca = sel_item(read_gene_data_file(file_ca_path))

245
1818


In [3]:
# import xlwt
# workbook = xlwt.Workbook()
# sheet_ca = workbook.add_sheet("Sheet_ca")
# for idx, item in enumerate(data_ca):
#     sheet_ca.write(idx, 0, '\n'.join(item))
# sheet_wnt4 = workbook.add_sheet("Sheet_wnt4")
# for idx, item in enumerate(data_wnt4):
#     sheet_wnt4.write(idx, 0, '\n'.join(item))
# workbook.save(r'基因摘要表格.xlsx')

### 生成Label Studio标记数据

In [4]:
import random
data_all = data_wnt4 + data_ca
random.seed(2345)
random.shuffle(data_all)
train_len = int(len(data_all) * 0.8)
train_data, test_data = data_all[:train_len], data_all[train_len:]

In [5]:
import json
from genLabeledData import *

gene_label_dict = {}
with open(file_gene_path) as f:
    for line in f.readlines():
        arr = line.strip().split('	')[1:]
        for e in arr:
            e = e.strip()
            if e != '':
                gene_label_dict[e] = 'Gene'

In [7]:
arr = [k for k, v in gene_label_dict.items()]
arr[:100]

['DEAD/H Box Polypeptide 11 Like 3',
 'DEAD/H Box Polypeptide 11 Like 1',
 'DEAD/H-Box Helicase 11 Like 1',
 'DEAD/H (Asp-Glu-Ala-Asp/His) Box Polypeptide 11 Like 1',
 'DDX11L3',
 'DEAD/H (Asp-Glu-Ala-Asp/His) Box Helicase 11 Like 1',
 'DDX11L9',
 'DDX11L1',
 'DEAD/H Box Polypeptide 11 Like 9',
 'DEAD/H-Box Helicase 11 Like 1 (Pseudogene)',
 'WAS Protein Family Homolog 7, Pseudogene',
 'WASH5P',
 'WASP Family Homolog 7, Pseudogene',
 'WASH7P',
 'WAS Protein Family Homolog 5 Pseudogene',
 'Family With Sequence Similarity 39, Member F',
 'FAM39F',
 'Hsa-MiR-6859-5p',
 'MI0022705',
 'MIR6859-1',
 'MIMAT0027618',
 'Hsa-Mir-6859-1',
 'MicroRNA Mir-6859-1',
 'MicroRNA 6859-1',
 'MIMAT0027619',
 'MIR1302-2HG',
 'Lnc-OR4F17-4',
 'HSALNG0000005',
 'NONHSAG024269.2',
 'MIR1302-2 Host Gene',
 'NONHSAG000003.2',
 'MicroRNA 1302-2',
 'MI0006363',
 'Hsa-MiR-1302',
 'Hsa-Mir-1302-2',
 'RF00951',
 'MIMAT0005890',
 'MIRN1302-2',
 'MIR1302-2',
 'Chromosome 1 F379 Retina Specific Protein',
 'F379',
 'NON

In [6]:
test_text_data = ['\n'.join(item) for item in test_data]
test_json_data = process_by_regex(gene_label_dict, test_text_data, language='EN')

with open(r'test_data.json', "w") as f:
    f.write(json.dumps(test_json_data, indent=1, separators=(',', ':'), ensure_ascii=False))

processing line: 1
time cost: 50.86302638053894
processing line: 2
time cost: 55.17097020149231
processing line: 3
time cost: 56.064029932022095
processing line: 4
time cost: 57.75397181510925
processing line: 5
time cost: 55.39799928665161
processing line: 6
time cost: 60.985029458999634
processing line: 7
time cost: 56.05196928977966
processing line: 8
time cost: 43.34500050544739
processing line: 9
time cost: 64.72300124168396
processing line: 10
time cost: 60.65499949455261
processing line: 11
time cost: 49.19297003746033
processing line: 12
time cost: 55.74174952507019
processing line: 13
time cost: 53.07337498664856
processing line: 14
time cost: 47.73800301551819
processing line: 15
time cost: 84.88799834251404
processing line: 16
time cost: 48.14700174331665
processing line: 17
time cost: 52.77799582481384
processing line: 18
time cost: 38.22300362586975
processing line: 19
time cost: 49.215970516204834
processing line: 20
time cost: 53.02002954483032
processing line: 21
time c

KeyboardInterrupt: 

In [7]:
train_text_data = ['\n'.join(item) for item in train_data]
train_json_data = process_by_regex(gene_label_dict, train_text_data, language='EN')

with open(r'train_data.json', "w") as f:
    f.write(json.dumps(train_json_data, indent=1, separators=(',', ':'), ensure_ascii=False))

In [17]:
### 文本转化为句子
data_all = data_wnt4 + data_ca
data_cls = []
for item in data_all:
    item_sent = []
    for line in item:
        arr = line.split('. ')
        for sent in arr:
            sent = sent.strip()
            if sent != '':
                if sent[-1] != '.':
                    item_sent.append(sent + '.')
                else:
                    item_sent.append(sent)
    data_cls.append(item_sent)



In [18]:
with open(r'D:\项目资料\基因表达\通路分析\gene_data_sentences.txt', 'w') as f:
    for item in data_cls:
        for sent in item:
            f.write('%s\n' % sent)

### 选择基因

In [6]:
word_list = []
for item in data_all:
    item_str = ' '.join(item)
    word_arr = item_str.replace(',', '').replace('.', '').split(' ')
    word_list.extend(list(set(word_arr)))
word_list = list(set(word_list))
print(len(word_list))
word_str = ' '.join(word_list)

37487


In [25]:
gr = '([ ]|^)' + 'NDRG1'.replace('.', '\.').replace('-', '\-') + '([ ,]|$)'
s = 'Rapid detection of the hypoxia-regulated CA-IX and NDRG1 gene expression in different glioblastoma cells in vitro.'
re.search(gr, word_str, re.I)

<re.Match object; span=(197401, 197408), match=' NDRG1 '>

In [28]:
data_gene_names = []
with open(file_gene_path) as f:
    for line in f.readlines():
        data_gene_names.append(line.strip())

gene_sel = []
for idx, gene_name in enumerate(data_gene_names):
    if idx % 100 == 0:
        print(idx)
    gene_name_regex = '([ ]|^)' + gene_name.replace('.', '\.').replace('-', '\-') + '([ ,]|$)'
    if re.search(gene_name_regex, word_str, re.I):
        print(gene_name, 'in')
        gene_sel.append(gene_name)
gene_sel.append('Carbonic anhydrase IX')

0
100
DVL1 in
PRDM16 in
200
300
ENO1 in
CA6 in
MIR34A in
CTNNBIP1 in
MTOR in
400
500
SDHB in
600
PADI4 in
700
WNT4 in
EPHB2 in
800
RHD in
ARID1A in
SFN in
900
1000
HPCA in
1100
GJB5 in
1200
RSPO1 in
1300
SLC2A1 in
CDC20 in
1400
PTPRF in
1500
CYP4B1 in
1600
LRP8 in
1700
C8B in
JUN in
1800
1900
WLS in
2000
DNAJB4 in
2100
2200
2300
F3 in
2400
COL11A1 in
2500
MYBPHL in
GSTM1 in
CSF1 in
2600
WNT2B in
RHOC in
2700
NRAS in
NGF in
2800
HSD3B1 in
NOTCH2 in
2900
PIAS3 in
TXNIP in
3000
3100
3200
S100A10 in
S100A11 in
CRNN in
S100A9 in
S100A6 in
3300
S100A4 in
S100A1 in
RAB13 in
SHE in
3400
MUC1 in
SYT11 in
RAB25 in
NES in
HDGF in
3500
PRCC in
CD1D in
AIM2 in
3600
CRP in
SDHC in
3700
RGS5 in
3800
F5 in
KIFAP3 in
3900
4000
TNR in
4100
LAMC2 in
4200
PDC in
4300
LHX9 in
ELF3 in
4400
REN in
4500
CD55 in
CD46 in
CD34 in
4600
HSD11B1 in
IRF6 in
ATF3 in
4700
TGFB2 in
4800
ENAH in
LEFTY2 in
4900
WNT9A in
WNT3A in
5000
RHOU in
RAB4A in
AGT in
5100
5200
FH in
AKT3 in
5300
NLRP3 in
5400
5500
5600
RRM2 in
ROC

49500
49600
TRIM16 in
49700
SHMT1 in
49800
49900
50000
FOXN1 in
50100
CPD in
50200
EVI2B in
50300
50400
TBC1D3 in
50500
ERBB2 in
TOP2A in
50600
50700
STAT5A in
STAT3 in
BRCA1 in
50800
FZD2 in
GFAP in
50900
WNT3 in
WNT9B in
51000
NGFR in
51100
COL1A1 in
TOB1 in
CA10 in
51200
MMD in
NOG in
COIL in
51300
CA4 in
51400
ACE in
PECAM1 in
51500
AXIN2 in
PRKCA in
51600
PRKAR1A in
SOX9 in
51700
FDXR in
51800
51900
CBX2 in
CBX4 in
52000
FASN in
SLC16A3 in
52100
52200
52300
52400
52500
IMPACT in
52600
CDH2 in
DSG2 in
TTR in
52700
52800
SMAD2 in
52900
SMAD4 in
DCC in
TCF4 in
53000
GRP in
PMAIP1 in
53100
BCL2 in
53200
53300
NFATC1 in
CSNK2A1 in
53400
PRNP in
53500
PCNA in
BMP2 in
PLCB1 in
JAG1 in
53600
53700
INSM1 in
FOXA2 in
53800
53900
ID1 in
BCL2L1 in
54000
E2F1 in
GDF5 in
54100
NFS1 in
TGM2 in
54200
54300
STK4 in
MMP9 in
CD40 in
54400
SNAI1 in
54500
BMP7 in
54600
GNAS in
54700
54800
PALM in
54900
TCF3 in
AMH in
55000
SIRT6 in
EBI3 in
55100
C3 in
55200
ANGPTL4 in
55300
55400
SPC24 in
EPOR in
5550

In [29]:
 'NDRG1' in gene_sel

True

In [30]:
print(len(gene_sel))
gene_sel = list(set(gene_sel))
print(len(gene_sel))
gene_sel=sorted(gene_sel, reverse=True)

768
767


In [9]:
print(gene_sel)

['WWOX-AS1', 'WWOX', 'VSTM2L', 'VDAC1P9', 'VAT1L', 'TTI1', 'TLDC2', 'TGM2', 'TGIF2-RAB5IF', 'TGIF2', 'SRC', 'SPTA1', 'SOGA1', 'SNORA71E', 'SNORA71D', 'SNORA71C', 'SNORA71B', 'SNORA71A', 'SNORA60', 'SNHG17', 'SNHG11', 'SMU1P1', 'SLC32A1', 'SLA2', 'SH2D2A', 'SDR42E1', 'SCAND1', 'SAMHD1', 'RPS3P7', 'RPS3P2', 'RPS3AP3', 'RPS27AP3', 'RPRD1B', 'RPN2', 'RPL7AP14', 'RPF2P1', 'ROMO1', 'RNU6-937P', 'RNU6-1191P', 'RNU4-40P', 'RNA5SP60', 'RNA5SP431', 'RN7SL612P', 'RN7SL237P', 'RN7SL156P', 'RN7SL116P', 'RN7SKP185', 'RN7SKP176', 'RN7SKP173', 'RBM39', 'RBL1', 'RALGAPB', 'RAD1P2', 'RAB5IF', 'PYHIN5P', 'PYHIN1', 'PRCC', 'PPP1R16B', 'PPIAP51', 'PPIAP50', 'PPIAP3', 'PLCG2', 'PKD1L2', 'PHF20', 'PEAR1', 'OR6Y1', 'OR6P1', 'OR6N2', 'OR6N1', 'OR6K6', 'OR6K5P', 'OR6K4P', 'OR6K3', 'OR6K2', 'OR6K1P', 'OR2AQ1P', 'OR10Z1', 'OR10X1', 'OR10T2', 'OR10T1P', 'OR10R3P', 'OR10R2', 'OR10R1P', 'OR10K2', 'OR10K1', 'OR10J9P', 'OR10J8P', 'OR10J7P', 'OR10J6P', 'OR10J5', 'OR10J4', 'OR10J3', 'OR10J2P', 'OR10J1', 'OR10AE1P', 'OR1

In [43]:
gene_sel.append('Carbonic anhydrase IX')
gene_sel.append('carbonic anhydrase IX')

### 拆分训练测试集

In [44]:
import random
random.shuffle(data_cls)
train_len = int(len(data_cls) * 0.8)
data_train, data_test = data_cls[:train_len], data_cls[train_len:]

In [38]:
len(data_train), len(data_test)

(1650, 413)

### 训练集分析

In [31]:
import re
gene_sel_trans = [gene.replace('.', '\.').replace('-', '\-') for gene in gene_sel]
gene_regex = '((([ ]|^)' + '([ .,]|$))|(([ ]|^)'.join(gene_sel_trans) + '([ .,]|$)))'


In [42]:

result = []
for item in data_train:
#     print(item)
    print('---------------------')
    item_result = []
    for line in item:
        line_match = re.findall(gene_regex, line)
        line_res = []
        if len(line_match) >= 2:
            for match in line_match:
                match_str = match[0].strip()
                match_str = match_str[:-1] if match_str[-1] in ['.', ','] else match_str
                line_res.append(match_str)
            line_res = list(set(line_res))
            if len(line_res) >= 2:
                item_result.append((line, line_res))
                print((line, line_res))
    result.append((item, item_result))

---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
(' Compared to normal adult colon, all mouse and human tumors over-expressed a large cluster of genes highly enriched for functional association to the control of cell cycle progression, proliferation, and migration, including those encoding MYC, AKT2, PLK1 and SPARC.', ['AKT2', 'SPARC', 'PLK1', 'MYC'])
(' Human CRC adenocarcinomas lost an additional suppressor module (IGFBP4, MAP4K1, PDGFRA, STAB1 and WNT4).', ['PDGFRA', 'STAB1', 'MAP4K1'])
(' Many human tumor samples also gained expression of a coordinately regulated module associated with advanced malignancy (ABCC1, FOXO3A, LIF, PIK3R1, PRNP, TNC, TIMP3 and VEGF).', ['TIMP3', 'PRNP', 'LIF', 'PIK3R1', 'TNC'])
---------------------
(' Genes overexpressed in breast carcinoma included TFF1, TFF3, FOXA1, CA12, GATA3, SDC1, PITX1, TH, EHFD1, EFEMP1, TOB1 and KLF2.', ['TFF3', 'KLF2', 'TH'

---------------------
(' The purpose of this study was to investigate mutations in 50 oncogenes and tumor suppressor genes, including APC and CTNNB1, in which mutations in COF have been previously reported.', ['APC', 'CTNNB1'])
('RESULTS: The expression assay revealed 12 differentially expressed Wnt/β-catenin pathway genes in COF, including the upregulation of CTNNB1, TCF7, NKD1, and WNT5 A, and downregulation of CTNNBIP1, FRZB, FZD6, RHOU, SFRP4, WNT10 A, WNT3 A, and WNT4, suggesting activation of the Wnt/β-catenin signaling pathway.', ['WNT4', 'NKD1', 'RHOU', 'CTNNBIP1', 'CTNNB1', 'FZD6', 'WNT3', 'FRZB', 'TCF7', 'SFRP4'])
(' NGS revealed 5 single nucleotide variants: TP53 (rs1042522), PIK3 CA (rs2230461), MET (rs33917957), KIT (rs3822214), and APC (rs33974176), but none of them was pathogenic.', ['TP53', 'MET', 'APC', 'KIT'])
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
------

---------------------
---------------------
(' Low levels of WNT2, WNT4, WNT5A have also been observed.', ['WNT4', 'WNT2', 'WNT5A'])
(' Decreased expression of WNT2 and WNT5A may be caused by miR-331-3p and miR-200b-5p, respectively.', ['WNT2', 'WNT5A'])
---------------------
---------------------
---------------------
---------------------
---------------------
(' Immunohistochemistry showed that atypical cells were positive for cytokeratin AE1&3, CK7, CD10, GATA3, glypican 3, Hep Par 1, carbonic anhydrase 9 (focal), and vimentin (focal) but negative for CK20, CD56, chromogranin A, synaptophysin, TTF1, HMB45, melan A, and PSA.', ['GATA3', 'TTF1'])
---------------------
---------------------
---------------------
---------------------
---------------------
(' A direct association of CA9 expression with epidermal growth factor receptor, c-erbB-2, and MUC1 expression was also noted (P < 0.', ['CA9', 'MUC1'])
---------------------
---------------------
---------------------
--------------

---------------------
(' Among these intersection genes, CA9, EBI3, SPOCK2, WDFY4, CD1D, and CCL22 were significantly correlated with overall survival in PDAC patients.', ['WDFY4', 'CD1D', 'CA9', 'SPOCK2', 'EBI3', 'CCL22'])
(' Moreover, multivariate Cox analysis revealed that CA9, SPOCK2, and CD1D were the most significant prognostic genes, and were closely correlated with immune infiltration in TCGA cohort.', ['CA9', 'CD1D', 'SPOCK2'])
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
(" Moreover, expression of four methylated genes (PLCXD3 (Phosphatidylinositol-Specific Phospholipase C, X Domain Containing 3), BAIAP2L2 (BAR/IMD Domain Containing Adaptor Protein 2 Like 2), NPR3 (Natriuretic Peptide Receptor 3), SNX10 (Sorting Nexin 10)) can influence patients' prognosis.", ['SNX10', 'NPR3', 'BAIAP2L2'])
(' Protein products of nine genes were identified as potential druggable targets

---------------------
---------------------
---------------------
(' Furthermore, inhibition of EGFR signaling intermediaries WNK1 and AKT1S1, members identified in the kinase array and previously unreported to play a role in the endometrium, also attenuate decidualization.', ['EGFR', 'AKT1S1', 'WNK1'])
---------------------
(' One tumor each had somatic FH mutation and SMARCB1 mutation.', ['FH', 'SMARCB1'])
---------------------
---------------------
---------------------
---------------------
---------------------
(' DNA was isolated from peripheral blood and used to genotype functional polymorphisms at the corresponding genes (HIF1A +1772 C\u2009>\u2009T, rs11549465; CA9\u2009+\u2009201 A\u2009>\u2009G; rs2071676; LOX +473\xa0G\u2009>\u2009A, rs1800449; KDR - 604\xa0T\u2009>\u2009C, rs2071559).', ['KDR', 'LOX'])
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
(' We validated 27 genes using qRT-PCR an

...

---------------------
---------------------
---------------------
(' The expression levels of DEC2, CA9, and SLC2A1 were significantly higher in the 4 atypical hyperplasia tissues and 82 endometrial carcinomas compared with those in the 21 normal endometria.', ['CA9', 'SLC2A1'])
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
(' In a total of 80 cytogenetically characterized fibroids from 50 patients, we were not only able to confirm the frequent occurrence of MED12 mutations but also to stratify two mutually exclusive pathways of leiomyomagenesis with either rearrangements of HMGA2 reflected by clonal chromosome abnormalities affecting 12q14~15 or by mutations affecting exon 2 of MED12.', ['HMGA2', 'MED12'])
---------------------
---------------------
---------------------
---------------------
(' By immunohistochemistry, tumor cells were negative for SDHB (11/11) and rarely SDHA 

(' We have used PeCa tissue arrays and multi-fluorophore labelled, quantitative, immunohistochemistry to interrogate the expression of WNT4, a Wnt ligand, and three targets of Wnt-ß-catenin transcription activation, namely, MMP7, cyclinD1 (CD1) and c-MYC in 141 penile tissue cores from 101 unique samples.', ['MMP7', 'WNT4'])
---------------------
---------------------
---------------------
---------------------
(' Both CA9 and CA12 were strongly induced by hypoxia in a range of tumor cell lines.', ['CA9', 'CA12'])
---------------------
---------------------
---------------------
---------------------
(' Positivity of MCT4 and PSPH was related to shorter disease-free survival, and SHMT1 was related to shorter overall survival (P < .', ['SHMT1', 'PSPH'])
(' Overexpression of MCT4, PSPH, and SHMT1 was associated with poorer prognosis.', ['SHMT1', 'PSPH'])
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
---

---------------------
---------------------
---------------------
---------------------
---------------------
(' In the majority of cases in which immunostaining was observed in meningiomas, staining was focal (involving <5% of neoplastic cells) in comparison with CA9 and CA10 immunostaining in renal cell carcinomas in which more than 50% of tumor cells stained the majority of cases.', ['CA9', 'CA10'])
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------


---------------------
(' Our results demonstrated that ALDH1, CD44, ARID1A, and CAIX in immunoreactive GA tumor cells exhibit different expression profiles compared with control cells and that these differences are associated with patient survival.', ['CD44', 'ARID1A'])
(' In addition, the loss of ALDH1, ARID1A, and CD44 and the overexpression of CAIX are important for tumor invasion and metastasis; therefore, they may serve as useful prognostic indicators of long-term survival in patients with GA.', ['ARID1A', 'CD44'])
---------------------
---------------------
---------------------
---------------------
(' Recently, we identified the tumour-associated carbonic anhydrases (CA), CA9 and CA12 as hypoxia-inducible in tumour cell lines.', ['CA9', 'CA12'])
---------------------
(' Linear regression analysis comparing FAZA retention with patterns of gene expression in individual tissue fragments revealed that CA9, GLUT1 and LOX mRNA levels were equally and strongly correlated to hypoxic ex

---------------------
(' Antibodies were used to quantify endogenous POR, MTRR, NDOR1, and NOS2A expression in 23 human tumor cell lines; however, only POR protein was detectable and its expression correlated with anoxic PR-104A reduction (r(2) = 0.', ['MTRR', 'NDOR1', 'POR'])
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
(' FBXL14 mRNA is decreased in tumors with a high expression of two proteins up-regulated in hypoxia, carbonic anhydrase 9 and TWIST1.', ['FBXL14', 'TWIST1'])
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
---------------------
('MATERIALS AND METHODS: Expression of the hypoxia-induced genes OPN, CA9, EPO, VEGF and HIF-1alpha was analyzed in three G

In [19]:
s='FSTX1	SFTX1	YSX1	FXX1	头晕	黄疸	PQTTX1	BJQX1	坐立不安	HBX1	YXFTX1	ZYXTX1	ZFTX1	YSFTX1	FSDFX1	CXFTX1	YHJZX1	ZFXTX1	突然发作	绞痛	JLFTX1	恶心呕吐	腹胀	TZPBX1	停经	YDLXX1	PNGBX1	FTSCFX1	YCFTX1	性别	年龄'

In [20]:
print(len(s.split('	')))

31
